In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from animal_shelter import AnimalShelter

# Username, password, and CRUD Python class instantiation
username = "aacuser"
password = "1234"
shelter = AnimalShelter()

# Populate dataframe
df = pd.DataFrame(list(shelter.read({})))

app = JupyterDash('SimpleExample')

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('Will McLaughlin - Project 2'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height': '10%', 'width': '10%'}
    ),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div([
            dcc.Dropdown(
                id='pie_dropdown',
                className='col s12 m6',
                options=[
                    {'label': 'Breed', 'value': 'breed'},
                    {'label': 'Age', 'value': 'age_upon_outcome'},
                    {'label': 'Outcome Type', 'value': 'outcome_type'}
                ],
                value='breed',
                multi=False,
                clearable=False
            )
        ], style={"width": "10%"}),
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={"width": "45%"}
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={"width": "45%"}
        )
    ])
])

# Update the data table based on filter type
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns'), Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'RESET':
        df = pd.DataFrame(list(shelter.read({})))
    elif filter_type == 'WATER':
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
    elif filter_type == 'MOUNTAIN OR WILDERNESS':
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
    else:
        df = pd.DataFrame(list(shelter.read({})))

    data = df.to_dict('records')
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    selected_rows = [0]
    
    return data, columns, selected_rows

# Update row styles when row is selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selectedRows):
    return [{
        'if': {'row_index': i},
        'background_color': '#D2F3FF'
    } for i in selectedRows]

# Update the pie chart based on dropdown selection
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"), Input('pie_dropdown', 'value')]
)
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [dcc.Graph(
        figure=px.pie(data_frame=dff, names=dropdownValue)
    )]

# Update the map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"), Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selectedRows):
    if not selectedRows:
        selected_row = 0
    else:
        selected_row = selectedRows[0]

    dff = pd.DataFrame.from_dict(viewData)
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    name = dff.iloc[selected_row, 9] if dff.iloc[selected_row, 9] else "No Name"

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX, posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX, posY], children=[
                dl.Tooltip(dff.iloc[selected_row, 4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]

app.run_server(debug=True) 






Dash app running on http://127.0.0.1:27971/
